In [1]:
import re
import pandas as pd
import json
from konlpy.tag import Mecab
#from tokenizers import BertWordPieceTokenizer
import matplotlib.pyplot as plt
import os

In [2]:
import nltk
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import urllib.request
from gensim.models import FastText
from glove import Corpus, Glove
import copy 

In [3]:
ls

CBOW_que_sample_44_type       SGNS_que_sample_44_type
FastText_que_sample_44_type   gloveque_sample_44_type.txt
MWP_Embedding_function.ipynb


In [4]:
df = pd.read_csv('../data/train_44_type_150000.csv', header = None)

In [6]:
df2 = pd.read_csv('../data/val_44_type_30000.csv', header = None)

In [33]:
df2

,0,1
0,"어떤 미지수에서 909를 뺐더니 73가 되었다면, 어떤 미지수는 얼마가 되겠습니까?",73 + 909
1,상현이가 책을 펼쳤는데 펼친 책의 두 쪽수의 합이 509이었다. 상현이가 펼친 책의...,((509-1)/2)+1
2,어떤 자연수에 35를 더한 후 28를 곱했습니다. 그 수에 30를 빼고 142로 나...,(((13*142)+30)/28)-35
3,자릿수가 세 개인 수의 덧셈식 'X5Z + 5Y0 = 942'에서 X에 해당하는 수...,(942 @ 100) - (500 @ 100) - (10-4+5+(10 - ((94...
4,616에 어떤 미지수를 나누어야 할 것을 잘못해서 30에 어떤 미지수를 나누었을 경...,616 * 825 / 30
...,...,...
29995,"J, I, K는 자연수일 때 J를 8로 나누면 몫은 I이며, 나머지는 K가 된다. ...",(8 - 1) * 8 + (8 - 1)
29996,어떤 미지수에서 427를 더한 결과로 746가 나왔습니다. 본래의 어떤 미지수의 값...,746 - 427
29997,"다영이는 스키 시합에서 861등을 했고, 상혁이는 859등을 했습니다. 윤호는 다영...",(859 + 861) / 2
29998,"4개의 숫자 3, 4, 2, 5를 한번씩만 사용하여 세 자리 수를 만들려고 합니다....",fact(4) / fact(4 - 3)


## 질문 전처리 / stopwords 미처리 버전 실행

In [30]:
list_que = []
mecab= Mecab()
for i in range(len(df2[0])):
    list_que.append(len(mecab.morphs(df[0][i])))

print(len(list_que))

30000


## Stopwords 처리

In [ ]:
mecab = Mecab()
list_que = []
stopwords = ['은','?','일까요','십시오','입니까','인가요',
             ',인지','한다면','가','다','에는','에서','기록', '순위',
             '앉아', '로', '.','시','오', '습니다','인지','한다','여라', '똑같은'] 
df[0] = df[0].apply(mecab.morphs)
df[0] = df[0].apply((lambda x : [item for item in x if item not in stopwords]))

for i in range(len(df[0])):
    list_que.append(df[0][i])

print(len(list_que))

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec): 
    """Callback to print loss after each epoch.""" 
    def __init__(self): 
        self.epoch = 0 
        self.loss_to_be_subed = 0 
    
    def on_epoch_end(self, model): 
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed 
        self.loss_to_be_subed = loss 
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now)) 
        self.epoch += 1

In [ ]:
def embedding_train(name, vocab_list):
    
    model = Word2Vec(sentences = vocab_list, vector_size=512, window= 6, min_count = 1, epochs = 30, 
                 compute_loss = True, workers = 4, sg= 0, hs = 1, callbacks =[callback()] )

    print('---- CBOW Word Embedding is over')

    model.wv.save_word2vec_format('CBOW_' + name)

    model = Word2Vec(sentences = vocab_list, vector_size=512, window= 6, min_count = 1, workers = 4, sg= 1,
                      negative = 10, compute_loss= True, hs = 1, epochs= 30, callbacks=[callback()])

    print('---- SGNS Word Embedding is over')


    model.wv.save_word2vec_format('SGNS_' + name)

    model = FastText(sentences = vocab_list, vector_size = 512, window= 6, min_count= 1, workers= 4, 
                     sg = 1, epochs = 30, hs= 1, callbacks =[callback()])

    print('---- FastText Word Embedding is over')


    model.wv.save_word2vec_format('FastText_' + name)

    corpus = Corpus()
    corpus.fit(vocab_list, window = 6)
    model = Glove(no_components= 512, learning_rate = 0.025)
    model.fit(corpus.matrix, no_threads = 4, epochs = 30, verbose = True)
    model.add_dictionary(corpus.dictionary)
    sample_dict = copy.deepcopy(model.dictionary)

    all_glove = [] ##
    i = 0

    for k in sample_dict.keys():
        m = model.word_vectors[model.dictionary[k]].tolist() ## 원래는 glove_model.word_vectors[glove_model.dictionary[k]] 로 실행해야함. 해당 단어 벡터list로 변경
        m.insert(0, k) ## list의 0번째에 해당단어 삽입
        m = " ".join(map(str, m)) ## 벡터값은 실수이고 단어는 str이라 join 타입이 맞지 않음 map 함수를 통해 
        all_glove.append(m)

    with open('glove' + name + '.txt' , 'w' ,encoding='UTF-8') as f: ## Glove 임베딩 벡터 저장
        for nam in all_glove:
            f.write(nam +'\n')

    print('---- GloVe Word Embedding is over')




## 문장형 문제의 단어 임베딩

In [ ]:
embedding_train('que_sample_44_type_512', list_que)

## 식 임베딩 (MWP Solver 설계에서는 필요하지 않음)

In [ ]:
embedding_train('Equ_sample', list_equ)

In [ ]:
load_equ = KeyedVectors.load_word2vec_format('FastText_Equ_sample')

In [ ]:
load_equ.most_similar(")")

In [ ]:
embedding = ['CBOW_Equ_sample','FastText_Equ_sample', 'SGNS_Equ_sample']
for i in embedding:
    print(i)
    load_equ = KeyedVectors.load_word2vec_format(i)
    print(load_equ.most_similar("("))

In [ ]:
embedding = ['CBOW_MWP_sample','FastText_MWP_sample', 'SGNS_MWP_sample']
for i in embedding:
    print(i)
    load_equ = KeyedVectors.load_word2vec_format(i)
    print(load_equ.most_similar("("))

In [ ]:
load_que = KeyedVectors.load_word2vec_format('FastText_MWP_sample')

In [ ]:
load_que.most_similar("다면")

In [ ]:
print(len(kor_que))
print(len(equ_))

In [ ]:
from collections import Counter
kor_que = Counter(np.hstack(list_que))
print(kor_que.most_common(30))

In [ ]:
print(kor_que.most_common(100))

In [ ]:
equ_ = Counter(np.hstack(list_equ))
print(equ_.most_common(20))

In [ ]:
corpus = Corpus()
corpus.fit(list_equ, window = 6)
model = Glove(no_components= 256, learning_rate = 0.025)
model.fit(corpus.matrix, no_threads = 4, epochs = 30, verbose = True)
model.add_dictionary(corpus.dictionary)
sample_dict = copy.deepcopy(model.dictionary)


model.most_similar("i")

In [ ]:
model.most_similar("+", number = 100)